# Linear Solver Performance: XDG Stokes, MPI-parallel
### Part 1, Benchmark Setup and Execution

In [ ]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.Utils.Geom;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.GridGen;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("LinslvPerfPar") ?? "LinslvPerfPar"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

In [ ]:
ExecutionQueues

In [ ]:
GetDefaultQueue()

## Utility definitions

In [ ]:
static class Utils {
    // DOF per cell in 3D for one variable
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

Note: The dimension of the domain $(-1,1)^3$ are assumed to be **centimeters**!
In this benchmark, realistic physical values are used, i.e. densities and viscosities of 
- water for Phase A (inside the droplet)
- air for Phase B (surounding)
The droplet is assumed to be in the millimeter range; therefore, all lenght-, area-, and volume-related properties 
must also be set in centimeters, e.g. the density of water is $10^{-3} \text{kg}/\text{cm}^3$.

In [ ]:
int[] Resolutions_3D = new int[] { 8, 16, 24, 32, 48, 64, 128, 256 };
string[] GridNameS = new string[Resolutions_3D.Length];
var ggcS = new (GridGenControl C, int MPIsize)[Resolutions_3D.Length];

for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] _xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] _yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] _zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (_xNodes.Length - 1)*(_yNodes.Length - 1)*(_zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-XdgStokes_J" + J);
    GridNameS[cnt] = GridName;  
    
    if(wmg.Grids.Where(grd => grd.Name.Contains(GridName)).Count() <= 0) {
        int NoOfProcs = (int) Math.Min(182, Math.Max(1, Math.Ceiling(J/200000.0)));
        Console.WriteLine("Must create: " + GridName + " with " + NoOfProcs + " processors.");
        
        var C = new GridGenControl();
        ggcS[cnt] = (C, NoOfProcs);
        C.SetDatabase(wmg.DefaultDatabase);
        
        C.GridName = GridName;

        C.GridBlocks = new GridGenControl.MeshBlock[] {
            new GridGenControl.Cartesian3D() {
                xNodes = _xNodes,
                yNodes = _yNodes,
                zNodes = _zNodes
            }
        };

        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -1-1e-8, -2, -2 }, new double[] { -1+1e-8, +2, +2 }), 
            "wall_left"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { +1-1e-8, -2, -2 }, new double[] { +1+1e-8, +2, +2 }), 
            "wall_right"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, -1-1e-8, -2 }, new double[] { +2, -1+1e-8, +2 }), 
            "wall_front"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, +1-1e-8, -2 }, new double[] { +2, +1+1e-8, +2 }), 
            "wall_back"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, -2, -1-1e-8 }, new double[] { +2, +2, -1+1e-8 }), 
            "wall_top"));
        C.BoundaryRegions.Add((
            new BoundingBox(new double[] { -2, -2, +1-1e-8 }, new double[] { +2, +2, +1+1e-8 }), 
            "wall_bottom"));
        
        
        C.SessionName = "GridCreation-" + GridName;
    } else {
        Console.WriteLine("Found grid: " + GridName);
    }
}

In [ ]:
foreach(var tt in ggcS) {
    if(tt.C != null) {
        Console.WriteLine(" Submitting: " + tt.C.SessionName); 
        var j = tt.C.CreateJob();
        j.RetryCount = 2;
        j.NumberOfMPIProcs = tt.MPIsize;
        j.Activate();
    }
}

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish
wmg.AllJobs

In [ ]:
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    int J = Res*Res*Res;
    
    Console.WriteLine("Searching for grid with " + J + " cells");
    grids[cnt] = wmg.Grids.FirstOrDefault(grd => grd.Name.Contains(GridNameS[cnt])); // grid must be present now
    
    if(grids[cnt] != null) {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");

        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    } else {
        Console.Error.WriteLine("missing: J = " + J);
    }
}

In [ ]:
grids

In [ ]:
NUnit.Framework.Assert.IsTrue(grids.All(g => g != null), "Unable to find/create some grid.");

## Setup Control Object for a Solver Run

In [ ]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
int[] PolyDegS = new int[] {2, 3 ,5};

In [ ]:
int[] MPIsizes = new int[] { 1, 2, 4, 8, 16, 32, 64, 128 };

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
var controls = new List<(XNSE_Control ctrl, int NoOfProcs)>();
LinearSolverCode solver_name = LinearSolverCode.exp_Kcycle_schwarz;
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {
foreach(int MPIsize in MPIsizes) {
    
    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3 + Np_p;
    if(J / MPIsize < 16) {
        // less than 16 cells per processor - to low for a multigrid.
        continue; 
    }
    if(DOF/MPIsize > 500000) {
        // not interested in doing more then 500'000 DOFs per processor
        continue;
    } 
    
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add((C, MPIsize));
       
    string caseName = string.Format("XdgStokes-J{0}_p{1}_Sz{2}", J, k, MPIsize);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    // Species A: Water; Species B: Air
    C.PhysicalParameters.rho_A             = 1.0; //1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.0; //1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1.0; //1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 1.0; //17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.Sigma             = 72.75e-3; // kg / sec²   
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Dont know
    // ============
    
    double r     = 0.5;
    double nonsp = 0.5;

    C.AddInitialValue("Phi", new Formula($"X => (X[0]/{r*nonsp}).Pow2() + (X[1]/{r}).Pow2() + (X[2]/{r}).Pow2() - 1", false));
    
    C.LSContiProjectionMethod = BoSSS.Solution.LevelSetTools.ContinuityProjectionOption.None;
    //C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.ComputeEnergyProperties = false;

    
    // Solver Stuff
    // ============
    
    C.LinearSolver           = solver_name.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.NoOfMultigridLevels = 100;
    C.TracingNamespaces = "BoSSS.Solution";
    C.LevelSet_ConvergenceCriterion     = 1e-6;

    //C.Option_LevelSetEvolution                          = LevelSetEvolution.FastMarching;
    C.AdvancedDiscretizationOptions.SST_isotropicMode   = SurfaceStressTensor_IsotropicMode.Curvature_Projected;
    //C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.Standard;
    C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.NoFilter;

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
    //C.dtFixed = 0.01;
}
}
}

Total number of simulations:

In [ ]:
controls.Count

In [ ]:
// Assert that the location of the fluid phases is as desired:
NUnit.Framework.Assert.Negative(controls[0].ctrl.InitialValues["Phi"].Evaluate(new double[]{0,0.0,0}, 0.0), "phase A (negative) must be inside");
NUnit.Framework.Assert.Positive(controls[0].ctrl.InitialValues["Phi"].Evaluate(new double[]{0,4.0,0}, 0.0), "phase B (positive) must be inside");

In [ ]:
// Basic checks on the multigrid configuration
foreach(var (ctrl, sz) in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [ ]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\Xstokes";
//foreach(var ctrl in controls) {
//    ctrl.savetodb = false;
//    ctrl.SaveToFile(System.IO.Path.Combine(path, ctrl.SessionName + ".obj"));
//}

## Launch Jobs

Use the default queue defined on this machine:

In [ ]:
ExecutionQueues

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
foreach((var ctrl, int MPIsize) in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    var j = ctrl.CreateJob();
    j.RetryCount = 1;
    j.NumberOfMPIProcs = MPIsize;
    j.Activate(myBatch);
    //ctrl.RunBatch();
}

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions.Where(sess => sess.Name.StartsWith("XdgStokes_J"))

In [ ]:
int succJob = wmg.AllJobs.Values.Where(job => !job.Name.Contains("GridCreation") && job.Status == JobStatus.FinishedSuccessful).Count();
int succSess = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes-J") && (Si.SuccessfulTermination == true)).Count();
(succJob, succSess)

In [ ]:
int failJob = wmg.AllJobs.Values.Where(job => !job.Name.Contains("GridCreation") && job.Status != JobStatus.FinishedSuccessful).Count();
int failSess = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes-J") && (Si.SuccessfulTermination == false)).Count();
(failJob, failSess)

In [ ]:
NUnit.Framework.Assert.AreEqual(succJob + failJob, controls.Count);
//NUnit.Framework.Assert.AreEqual(succJob, succSess);
//NUnit.Framework.Assert.AreEqual(failJob, failSess);

In [ ]:
var NoSuccess = wmg.AllJobs.Values
                        .Where(job => job.Status != JobStatus.FinishedSuccessful)
                        .ToArray();
NoSuccess

In [ ]:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + ((fail.LatestDeployment?.DeploymentDirectory?.FullName) ?? " no deployment directory"));

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes") && !Si.Name.Contains("GridCreation") &&
                                        (Si.SuccessfulTermination == false
                                        || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0));
FailedSessions